##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Keras Tuner の基礎

<table class="tfo-notebook-buttons" align="left">
  <td> <img src="https://www.tensorflow.org/images/tf_logo_32px.png"><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ja/tutorials/keras/keras_tuner.ipynb">TensorFlow.org で表示</a> </td>
  <td> <img src="https://www.tensorflow.org/images/colab_logo_32px.png"><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ja/tutorials/keras/keras_tuner.ipynb">Google Colab で実行</a> </td>
  <td> <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png"><a target="_blank" href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ja/tutorials/keras/keras_tuner.ipynb">GitHub でソースを表示</a> </td>
  <td> <img src="https://www.tensorflow.org/images/download_logo_32px.png"><a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/keras_tuner.ipynb">ノートブックをダウンロード</a> </td>
</table>

## 概要

Keras Tuner は、TensorFlow プログラム向けに最適なハイパーパラメータを選択するためのライブラリです。ユーザーの機械学習（ML）アプリケーションに適切なハイパーパラメータを選択するためのプロセスは、*ハイパーパラメータチューニング*または*ハイパーチューニング*と呼ばれます。

ハイパーパラメータは、ML のトレーニングプロセスとトポロジーを管理する変数です。これらの変数はトレーニングプロセス中、一貫して定数を維持し、ML プログラムのパフォーマンスに直接影響を与えます。ハイパーパラメータには、以下の 2 種類があります。

1. **モデルハイパーパラメータ**: 非表示レイヤーの数と幅などのモデルの選択に影響します。
2. **アルゴリズムハイパーパラメータ**: 確率的勾配降下法 (SGD) の学習率や k 最近傍 (KNN) 分類器の最近傍の数など、学習アルゴリズムの速度と質に影響します。

このチュートリアルでは、Keras Tuner を使用して、画像分類アプリケーションのハイパーチューニングを実施します。

## セットアップ

In [ ]:
import tensorflow as tf
from tensorflow import keras

Keras Tuner をインストールしてインポートします。

In [ ]:
!pip install -q -U keras-tuner

In [ ]:
import keras_tuner as kt

## データセットをダウンロードして準備する

このチュートリアルでは、Keras Tuner を使用して、[Fashion MNIST データセット](https://github.com/zalandoresearch/fashion-mnist)の服飾の画像を分類する学習モデル向けに最適なハイパーパラメータを見つけます。

データを読み込みます。

In [ ]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [ ]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## モデルを定義する

ハイパーチューニングを行うモデルを構築する際、モデルアーキテクチャのほかにハイパーパラメータ検索空間も定義します。ハイパーチューニング用にセットアップするモデルを*ハイパーモデル*と呼びます。

ハイパーモデルの定義は、以下の 2 つの方法で行います。

- モデルビルダー関数を使用する
- Keras Tuner API の `HyperModel` クラスをサブクラス化する

また、コンピュータビジョンアプリケーション用の [HyperXception](https://keras-team.github.io/keras-tuner/documentation/hypermodels/#hyperresnet-class) と <a>HyperResNet</a> という 2 つの事前定義済みの <code>HyperModel</code> クラスも使用します。

このチュートリアルでは、モデルビルダー関数を使用して、画像分類モデルを定義します。モデルビルダー関数は、コンパイル済みのモデルを返し、インラインで定義するハイパーパラメータを使用してモデルをハイパーチューニングします。

In [ ]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## チューナーをインスタンス化してハイパーチューニングを実行する

チューナーをインスタンス化して、ハイパーチューニングを実行します。Keras Tuner には、`RandomSearch`、`Hyperband`、`BayesianOptimization`、および `Sklearn` チューナーがあります。このチュートリアルでは、[Hyperband](https://arxiv.org/pdf/1603.06560.pdf) チューナーを使用します。

Hyperband チューナーをインスタンス化するには、ハイパーモデル、最適化する `objective`、およびトレーニングするエポックの最大数 (`max_epochs`) を指定する必要があります。

In [ ]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

Hyperband チューニングアルゴリズムは、適応型リソース割り当てと早期停止を使用して、高パフォーマンスモデルに素早く収束させます。これは、トーナメント式のツリーを使用して行われます。アルゴリズムは、数回のエポックで大量のモデルをトレーニングし、性能の高い上位半数のモデル次のラウンドに持ち越します。Hyperband は、1 + log<sub><code>factor</code></sub>(`max_epochs`) を計算し、直近の整数に繰り上げて、トーナメントでトレーニングするモデル数を決定します。

検証損失の特定の値に達した後、トレーニングを早期に停止するためのコールバックを作成します。

In [ ]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

ハイパーパラメータ検索を実行します。検索メソッドの引数は、上記のコールバックのほか、`tf.keras.model.fit` に使用される引数と同じです。

In [ ]:
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

## モデルをトレーニングする

検索から取得したハイパーパラメータを使用してモデルをトレーニングするための最適なエポック数を見つけます。

In [ ]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

ハイパーモデルを再インスタンス化し、前述の最適なエポック数でトレーニングします。

In [ ]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

このチュートリアルを終了するには、テストデータでハイパーモデルを評価します。

In [ ]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

`my_dir/intro_to_kt` ディレクトリには、ハイパーパラメータ検索中に実行された各トライアル（モデル構成）の詳細なログとチェックポイントが含まれます。ハイパーパラメータ検索を再実行する場合、Keras Tuner は、これらのログの既存の状態を使用して、検索を再開します。この動作を無効にするには、チューナーをインスタンス化する際に、`overwrite = True` 引数を追加で渡してください。

## まとめ

このチュートリアルでは、Keras Tuner の使用して、モデルのハイパーパラメータを調整する方法を学習しました。Keras Tuner の調査委については、以下のその他のリソースをご覧ください。

- [TensorFlow ブログの KerasTuner](https://blog.tensorflow.org/2020/01/hyperparameter-tuning-with-keras-tuner.html)
- [KerasTuner の Web サイト](https://keras-team.github.io/keras-tuner/)

また、モデルのハイパーパラメータを対話式で調整できる、TensorBoard の [HParams Dashboard](https://www.tensorflow.org/tensorboard/hyperparameter_tuning_with_hparams) もご覧ください。